## import包

In [1]:
import sys
sys.path.append('../AutoX')

from autox import AutoXRecommend

/home/caihengxing/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not mo

In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

## 读取数据

In [3]:
path = '/home/caihengxing/kaggle/h_and_m/input/'

In [4]:
os.listdir(path)

['customers.csv',
 'transactions_train.csv',
 'articles.csv',
 'sample_submission.csv']

In [5]:
inter_df = pd.read_csv(path + 'transactions_train.csv', dtype={'article_id': str})
user_df = pd.read_csv(path + 'customers.csv')
item_df = pd.read_csv(path + 'articles.csv', dtype={'article_id': str})
sample_submission = pd.read_csv(path + 'sample_submission.csv')

In [6]:
inter_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


## 配置参数

In [7]:
uid = 'customer_id'
iid = 'article_id'
time_col = 't_dat'
recall_num = 12

## 注意时间列需要转成时间格式

In [8]:
inter_df[time_col] = pd.to_datetime(inter_df[time_col])

In [9]:
inter_df[time_col].min(), inter_df[time_col].max()

(Timestamp('2018-09-20 00:00:00'), Timestamp('2020-09-22 00:00:00'))

## 执行AutoXRecommend

In [10]:
autoXRecommend = AutoXRecommend()

autoXRecommend.fit(inter_df = inter_df, user_df = user_df, item_df = item_df,
                  uid = uid, iid = iid, time_col = time_col,
                  recall_num = recall_num, 
                  mode = 'recalls', recall_method = 'popular')

In [11]:
test_users = list(sample_submission[uid])

In [12]:
res = autoXRecommend.transform(test_users)

100%|██████████| 1371980/1371980 [00:12<00:00, 112111.86it/s]


## 格式处理

In [13]:
res['prediction'] = res['prediction'].apply(lambda x: ' '.join(x))

## 结果查看

In [14]:
print(res.shape)
res.head()

(1371980, 2)


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0924243001 0924243002 0923758001 0918522001 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0924243002 0923758001 0918522001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0924243001 0924243002 0923758001 0918522001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0924243002 0923758001 0918522001 09...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0924243001 0924243002 0923758001 0918522001 09...


In [15]:
print(sample_submission.shape)
sample_submission.head()

(1371980, 2)


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


## 保存结果

In [16]:
import os
path_output = './hm_sub'
os.makedirs(path_output, exist_ok = True)

In [17]:
res.to_csv(f'{path_output}/AutoX_popular_recall.csv', index = False)

In [18]:
!zip -r ./hm_sub/AutoX_popular_recall.csv.zip ./hm_sub/AutoX_popular_recall.csv

  adding: hm_sub/AutoX_popular_recall.csv (deflated 80%)
